In [1]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

# For importing .xml files
# import xml.etree.ElementTree as ET

# For handeling dataframes
import pandas as pd
import numpy as np
import re                                  # For regular expression operations
import string                              # For string operations
import math

# For text preprocessing
import nltk                                # Natural Language Toolkit
from nltk.corpus import stopwords          # For stop words that come with NLTK

# from nltk.stem import PorterStemmer as ps  # For stemming

# For building neural netwrok models
import sklearn
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam

# For model evaluation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
from sklearn.metrics import roc_auc_score
import time
import random

# For text agumenters
import os
os.environ["MODEL_DIR"] = '../model'
import nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action

import gc
# import tqdm
# from tqdm import tqdm # To show the progress bar
# from joblib import Parallel, delayed

nltk.download('averaged_perceptron_tagger')  # 
nltk.download('wordnet')                     # For Synonym augmenter
nltk.download('punkt')                       # For BackTranslation augmenter

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# pip install torch torchvision

In [3]:
# pip install nlpaug

In [4]:
# import sys

In [5]:
labels_all = pd.read_csv("data/labels_all.csv")
labels_all.drop(['Unnamed: 0'], axis=1, inplace=True)

text_all = pd.read_csv('data/text_all.csv')
text_all.drop(['Unnamed: 0'], axis=1, inplace=True)

#  Create binary labels for id
disease_types = ['Hypertriglyceridemia','Venous Insufficiency','Asthma','Gout','OSA','PVD','Gallstones','OA','GERD',
                'Depression','Obesity','CHF','Hypercholesterolemia','CAD','Diabetes','Hypertension']

In [6]:
# Make the outcome variables (disease types) binary
def make_binary(labels_df):
    for i in range(16):
        # In the disease column, code the disease of interest as 1 and other disease types as 0 to indicate 
        # whether this row is about this particular disease
        labels_df["disease_"+str(i)] = [1 if x==disease_types[i] else 0 for x in labels_df["disease"]]
        # In the label column, code Y as 1 and everything else as 0 to indicate whether a disease is present
        # (it may or may not be about this particular disease)
        labels_df["label_"+str(i)] = [1 if x=="Y" else 0 for x in labels_df["label"]] 
        # Create a "This_Disease" column that indicates whether a patient has this particular disease or not
        # A patient is coded as having this disease only when (disease is 1) AND (label is 1)
        labels_df["This_Disease_"+str(i)] = labels_df["disease_"+str(i)] * labels_df["label_"+str(i)] 
        labels_df.drop(["disease_"+str(i), "label_"+str(i)], axis=1, inplace=True)

    # Now that all useful info in "disease" and "Label" are combined in column "This_Disease", no longer need disease and label
    labels_df.drop(['label', 'disease'], axis=1, inplace=True)
    # Duplicates after removing disease and label are due to 0's in the "disease" column if the patient had other disease info
    labels_df.drop_duplicates(inplace = True)
    # Aggregate the binary values into one row for each id
    labels_df = labels_df.groupby('id').aggregate('sum').reset_index()
    
    return(labels_df)

In [7]:
binary_labels = make_binary(labels_all)

# Calculate the disease prevalence for each disease type
prevalences = (binary_labels.aggregate('sum')/1237).iloc[1:,]

# Combine the text data and the labels
df = text_all.merge(binary_labels, on="id", how="left")

In [8]:
prevalences

This_Disease_0     0.054163
This_Disease_1     0.067098
This_Disease_2     0.132579
This_Disease_3     0.129345
This_Disease_4     0.139046
This_Disease_5     0.145513
This_Disease_6     0.151981
This_Disease_7     0.181083
This_Disease_8     0.198868
This_Disease_9     0.201293
This_Disease_10    0.403395
This_Disease_11    0.433306
This_Disease_12    0.465643
This_Disease_13    0.551334
This_Disease_14    0.663703
This_Disease_15    0.729992
dtype: float64

In [9]:
########################################################################
# NLP Preprocessing
########################################################################
# Download the stopwords from NLTK
nltk.download('stopwords')
stopwords_english = stopwords.words('english') 
additional_sw = ['admission', 'date', 'discharge', 'summary', 'diagnoses', 'diagnosis', 'history',
                    'patient', 'report', 'status', 'signed', 'attending', 'service', 'dictated', 'dictation']
all_stopwords = additional_sw + stopwords_english

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# Clean text
def preprocessing(text):
    text = "".join([x for x in text if x not in string.punctuation]) # Remmove punctuations 
    text = "".join([x.lower() for x in text]) # Convert to lower case
    text = ' '.join(['' if (x in all_stopwords) else x for x in text.split()]) # Remove stopwords
    text = re.sub("(\W|\d+|\n)", " ", text).strip() # remove spaces, digits and line breaks
    text = ' '.join(['' if (len(x) <= 2) else x for x in text.split()]) # Remove short words
    return(text)

In [11]:
#######################################################################
# Functions
#######################################################################

# Define a function to evaluate the model performance in terms of F1 score
def evaluate(model, X, y):
#     pred = model.predict_classes(X)
    pred = (model.predict(X) > 0.5).astype("int32")[:,1]
    acc = np.sum(y == pred)/len(pred)
    
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()

    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = (2*precision*recall)/(precision + recall)
    
    y_pred = model.predict(X)
    auc_roc = round(roc_auc_score(y, y_pred[:,1]),4)
    pre, rec, thresholds = precision_recall_curve(y, y_pred[:,1])
    auc_pr = round(auc(rec, pre),4)
    return(auc_roc, auc_pr, acc, precision, recall, specificity, f1)

In [12]:
def _tokenizer(text, token_pattern=r"(?u)\b\w\w+\b"):
    token_pattern = re.compile(token_pattern)
    return token_pattern.findall(text)

In [13]:
# Arguments for character augmenter
aug_min_1=200
aug_max_1=2000
aug_p=0.3 

# Arguments for word augmenter
aug_min_2=100
aug_max_2=1000
top_k=500 

# Define the augmenters
# Character Augmenters
augs = []
augs.append(nac.KeyboardAug(aug_char_min=aug_min_1, aug_char_max=aug_max_1,
                         aug_char_p = aug_p, aug_word_min=aug_min_2, aug_word_max=aug_max_2,
                         include_special_char=False,
                         include_upper_case=False, include_numeric=False)) # Substitute characters by keyboard distance
augs.append(nac.RandomCharAug(action="insert", aug_char_min=aug_min_1, aug_char_max=aug_max_1,
                         aug_char_p = aug_p, aug_word_min=aug_min_2, aug_word_max=aug_max_2,
                         include_upper_case=False, include_numeric=False)) # Insert characters randomly
augs.append(nac.RandomCharAug(action="substitute", aug_char_min=aug_min_1, aug_char_max=aug_max_1,
                         aug_char_p = aug_p, aug_word_min=aug_min_2, aug_word_max=aug_max_2,
                         include_upper_case=False, include_numeric=False)) # Substitute characters randomly
augs.append(nac.RandomCharAug(action="swap", aug_char_min=aug_min_1, aug_char_max=aug_max_1,
                         aug_char_p = aug_p, aug_word_min=aug_min_2, aug_word_max=aug_max_2,
                         include_upper_case=False, include_numeric=False)) # Swap characters randomly
augs.append(nac.RandomCharAug(action="delete", aug_char_min=aug_min_1, aug_char_max=aug_max_1,
                         aug_char_p = aug_p, aug_word_min=aug_min_2, aug_word_max=aug_max_2,
                         include_upper_case=False, include_numeric=False)) # Delete characters randomly
# Word Augmenter
augs.append(naw.RandomWordAug(action="swap", aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Swap words randomly
augs.append(naw.RandomWordAug('delete', aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Delete words randomly
augs.append(naw.RandomWordAug(action='crop', aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Delete a set of contunous words randomly
augs.append(naw.SynonymAug(aug_src='wordnet', aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Substitute words by WordNet's synonym
augs.append(naw.AntonymAug(aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Substitute word by antonym
augs.append(naw.SplitAug(aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Split words to two tokens randomly
augs.append(naw.SpellingAug(aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Substitute words by simulated spelling errors
#     aug_13 = naw.WordEmbsAug(model_type='word2vec', 
#                             model_path=model_dir+'GoogleNews-vectors-negative300.bin',
#                             action="insert") # Insert word randomly by word embeddings similarity
#     aug_14 = naw.WordEmbsAug(model_type='word2vec', 
#                             model_path=model_dir+'GoogleNews-vectors-negative300.bin',
#                             action="substitute") # Substitute word randomly by word embeddings similarity
# Contextual Word Embeddings Augmenter
augs.append(naw.ContextualWordEmbsAug(model_path='bert-base-uncased', top_k=None, 
                                  action="insert", aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Insert word by contextual word embeddings (Bert)
augs.append(naw.ContextualWordEmbsAug(model_path='bert-base-uncased', top_k=None, 
                                   action="substitute", aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Substitute word by contextual word embeddings (Bert)
augs.append(naw.ContextualWordEmbsAug(model_path='distilbert-base-uncased', top_k=None, 
                                   action="insert", aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Substitute word by contextual word embeddings (DistilBERT)
augs.append(naw.ContextualWordEmbsAug(model_path='distilbert-base-uncased', top_k=None, 
                                   action="substitute", aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Substitute word by contextual word embeddings (DistilBERT)
augs.append(naw.ContextualWordEmbsAug(model_path='roberta-base', top_k=None, 
                                   action="insert", aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Substitute word by contextual word embeddings (RoBERTA)
augs.append(naw.ContextualWordEmbsAug(model_path='roberta-base', top_k=None, 
                                   action="substitute", aug_min=aug_min_2, aug_max=aug_max_2, aug_p=aug_p)) # Substitute word by contextual word embeddings (RoBERTA)
# Abstractive Summarization Augmenter
# augs.append(nas.AbstSummAug(model_path='t5-base',min_length=300,max_length=1000))
# Back Translation Augmenter
# augs.append(naw.BackTranslationAug(from_model_name='facebook/wmt19-en-de', # English to German
#                              to_model_name='facebook/wmt19-de-en', # German to English
#                              batch_size=32, max_length=2000))
# # TFIDF Augmenter
# augs.append(naw.TfIdfAug(model_path='.', tokenizer=_tokenizer, action='insert',
#                               aug_min=aug_min_2,aug_max=aug_max_2, aug_p=aug_p, top_k=top_k))
# # TFIDF Augmenter
# augs.append(naw.TfIdfAug(model_path='.', tokenizer=_tokenizer, action='substitute',
#                               aug_min=aug_min_2,aug_max=aug_max_2, aug_p=aug_p, top_k=top_k))

In [14]:
len(augs)

18

In [15]:
# pip install transformers

In [16]:
# Create the Transformer model
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_length, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.pos_emb = layers.Embedding(input_dim=max_length, output_dim=embedding_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_length, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.3):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embedding_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    

def transformer_model(vocab_size, embedding_dim,max_length,dropout_rate):   
    inputs = layers.Input(shape=(max_length,))
    embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embedding_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embedding_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(2, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    opt = tf.keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    
    return(model)

In [17]:
#######################################################################
# Modeling
#######################################################################

# Set the number of epochs and the batch size
epochs = 20 ###################################################################
# epochs = 2
batch_size = 32
max_length = 557
dropout_rate = 0.3
embedding_dim = 200
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

time_1 = time.time()

X = df['text']
# X = df['text'][:50]
with open('other/DA_until_balanced_Transformer.csv','a') as fd:
    fd.write(f'DA until balanced_Hypertension_5 iterations\n')
for i in [13,14,15]:

    event_categorical = to_categorical(df['This_Disease_'+str(i)])
#     event_categorical = to_categorical(df['This_Disease_'+str(i)][:50])
    prevalence = prevalences.iloc[i]
    dis_type = disease_types[i]
    j = 0 # to keep track of the iteration number
    time_start = time.time()
    
    # Run the model 10 times with a different split each time  
#     sss = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=0)##################################
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=0)##################################
    for train_index, test_index in sss.split(X, event_categorical):
        time_s = time.time()
        
        j += 1
        iteration = "iter" + str(j)
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = event_categorical[train_index], event_categorical[test_index]
        
        # Select the samples to be augmented 
        if prevalence < 0.5: # If disease presence is the minority
            x_aug_0 = x_train[y_train[:,1]==1] # Augment disease presence cases
            y_aug_0 = y_train[y_train[:,1]==1,0]

        else: # If disease absence is the minority
            x_aug_0 = x_train[y_train[:,0]==1] # Augment disease absence cases
            y_aug_0 = y_train[y_train[:,0]==1,0]

        x_aug_0 = np.array(x_aug_0)

        # Randomly augment a predetermined number of the selected cases
        length = len(x_aug_0) # Total number of texts availble for aumentation
        num = math.ceil(abs(prevalence - 0.5)*len(x_train))*2 # Number of samples needed to balance the classes in the training set
        if num <= length:
            # If the number of texts to be augmented is larger than the number of samples needed
            samples = np.random.choice(range(length), size=num, replace=False) # Generate random indices without replacement
        else:
            samples = np.random.choice(range(length), size=num, replace=True) # Generate random indices with replacement
                
        # Augment the training set             
        # Go through all 17 augmenters: (run Synonym, TFIDF, and Backtranslation separately)
#         for k in range(3):
        for k in range(len(augs)):
            DA_start = time.time()
            aug = augs[k] # Assign the kth augmenter

            new_text = []
            new_y_0 = []
            new_y_1 = []

            for ii in samples: # Go through each sample to be augmented   
                text = x_aug_0[ii]
                label_aug = y_aug_0[ii]
                augmented_text = aug.augment(text)
                    
                new_text.append(augmented_text)
                new_y_0.append(label_aug) # Assign the label of the original text to the augmented text
                new_y_1.append(1-label_aug) # Assign the label of the original text to the augmented text

            # Append the augmented texts and their corresponding labels to the original data
            x_aug = np.append(x_train, new_text)
            y_0 = np.append(y_train[:,0], new_y_0)
            y_1 = np.append(y_train[:,1], new_y_1)
            y_aug = (np.vstack((y_0,y_1))).T
    
            # Shuffle the order of the augmented dataset
            augmented = pd.DataFrame((np.vstack((x_aug, y_aug[:,0], y_aug[:,1]))).T)
            augmented = augmented.sample(frac=1).reset_index(drop=True)
            
            DA_time = time.time() - DA_start
            augmenter = "aug_"+str(k)
            with open('other/DA_until_balanced_Transformer.csv','a') as fd:
                fd.write(f'{dis_type},{iteration},{augmenter},DA Running Time:,{DA_time}\n')
    
            # Convert the dataframe to numpy arrays
            x_train_1 = np.array(augmented.iloc[:,0])
            y_train_1 = np.array(augmented.iloc[:,[1,2]])
            y_train_1 = y_train_1.astype('float32')
            
            
            # Run CNN model
            model_time_start = time.time()
            
            # Preprocess the text
            vfunc = np.vectorize(preprocessing)
            x_train_1 = vfunc(x_train_1)
            x_test_1 = vfunc(x_test)

            # Tokenize the text   
            tokenizer = Tokenizer(num_words=5000) # get the frequency of all tokens and use the 5000 most common ones
            tokenizer.fit_on_texts(x_train_1)
            x_train_1 = tokenizer.texts_to_sequences(x_train_1)
            x_test_1 = tokenizer.texts_to_sequences(x_test_1)
            vocab_size = len(tokenizer.word_index) + 1 # plus the reserved index 0
            word_index = tokenizer.word_index

            # Pad the sequences with 0's
            x_train_1 = pad_sequences(x_train_1, padding='post', maxlen=max_length) 
            x_test_1 = pad_sequences(x_test_1, padding='post', maxlen=max_length)

            # Fit the Transformer model
            mymodel = transformer_model(vocab_size, embedding_dim,max_length,dropout_rate)
            mymodel.fit(x_train_1, y_train_1, epochs=epochs, batch_size=batch_size)

            # Collect and log evaluation metrics
            auc_roc, auc_pr, acc, precision, recall, specificity, f1 = evaluate(mymodel, x_test_1, y_test[:,1])
            model_time = time.time() - model_time_start

            with open('other/DA_until_balanced_Transformer.csv','a') as fd:
                fd.write(f'{dis_type},{iteration},{augmenter},{auc_roc},{auc_pr},{acc},{precision},{recall},{specificity},{f1},{model_time},{len(word_index)},{len(x_aug)}\n')
            
            del tokenizer, mymodel, vocab_size, word_index, x_aug, y_aug, augmented, x_train_1, x_test_1, y_train_1
            gc.collect()
        
        
        time_e = time.time() - time_s
        with open('other/DA_until_balanced_Transformer.csv','a') as fd:
            fd.write(f'{dis_type},1 iteration 18 DA,{time_e}\n')

    running_time = time.time() - time_start
    with open('other/DA_until_balanced_Transformer.csv','a') as fd:
            fd.write(f'{dis_type},10 iteration training time,{running_time}\n')

Epoch 1/20
32/32 [==============================] - 28s 848ms/step - loss: 0.6753 - auc: 0.6396
Epoch 2/20
32/32 [==============================] - 27s 847ms/step - loss: 0.4275 - auc: 0.8885
Epoch 3/20
32/32 [==============================] - 28s 860ms/step - loss: 0.2192 - auc: 0.9702
Epoch 4/20
32/32 [==============================] - 27s 854ms/step - loss: 0.0835 - auc: 0.9950
Epoch 5/20
32/32 [==============================] - 27s 856ms/step - loss: 0.0260 - auc: 0.9987
Epoch 6/20
32/32 [==============================] - 27s 856ms/step - loss: 0.0139 - auc: 0.9989
Epoch 7/20
32/32 [==============================] - 27s 855ms/step - loss: 0.0093 - auc: 0.9989
Epoch 8/20
32/32 [==============================] - 27s 853ms/step - loss: 0.0078 - auc: 1.0000
Epoch 9/20
32/32 [==============================] - 27s 856ms/step - loss: 9.2576e-04 - auc: 1.0000
Epoch 10/20
32/32 [==============================] - 27s 852ms/step - loss: 4.2035e-04 - auc: 1.0000
Epoch 11/20
32/32 [============

32/32 [==============================] - 29s 900ms/step - loss: 0.1127 - auc: 0.9907
Epoch 5/20
32/32 [==============================] - 29s 897ms/step - loss: 0.0511 - auc: 0.9976
Epoch 6/20
32/32 [==============================] - 29s 900ms/step - loss: 0.0328 - auc: 0.9985
Epoch 7/20
32/32 [==============================] - 29s 893ms/step - loss: 0.0140 - auc: 0.9989
Epoch 8/20
32/32 [==============================] - 29s 902ms/step - loss: 0.0070 - auc: 0.9994
Epoch 9/20
32/32 [==============================] - 29s 894ms/step - loss: 0.0086 - auc: 1.0000
Epoch 10/20
32/32 [==============================] - 29s 897ms/step - loss: 0.0011 - auc: 1.0000
Epoch 11/20
32/32 [==============================] - 29s 896ms/step - loss: 4.4099e-04 - auc: 1.0000
Epoch 12/20
32/32 [==============================] - 29s 901ms/step - loss: 3.0263e-04 - auc: 1.0000
Epoch 13/20
32/32 [==============================] - 29s 899ms/step - loss: 2.6362e-04 - auc: 1.0000
Epoch 14/20
32/32 [================

32/32 [==============================] - 30s 938ms/step - loss: 0.0317 - auc: 0.9986
Epoch 8/20
32/32 [==============================] - 30s 939ms/step - loss: 0.0124 - auc: 0.9989
Epoch 9/20
32/32 [==============================] - 30s 944ms/step - loss: 0.0076 - auc: 0.9990
Epoch 10/20
32/32 [==============================] - 30s 944ms/step - loss: 0.0061 - auc: 1.0000
Epoch 11/20
32/32 [==============================] - 30s 940ms/step - loss: 0.0042 - auc: 1.0000
Epoch 12/20
32/32 [==============================] - 30s 938ms/step - loss: 0.0098 - auc: 0.9999
Epoch 13/20
32/32 [==============================] - 30s 941ms/step - loss: 0.0237 - auc: 0.9997
Epoch 14/20
32/32 [==============================] - 30s 940ms/step - loss: 0.0238 - auc: 0.9987
Epoch 15/20
32/32 [==============================] - 30s 943ms/step - loss: 0.0314 - auc: 0.9986
Epoch 16/20
32/32 [==============================] - 30s 939ms/step - loss: 0.0084 - auc: 1.0000
Epoch 17/20
32/32 [=========================

32/32 [==============================] - 29s 906ms/step - loss: 0.0096 - auc: 0.9989
Epoch 11/20
32/32 [==============================] - 29s 908ms/step - loss: 0.0029 - auc: 1.0000
Epoch 12/20
32/32 [==============================] - 29s 910ms/step - loss: 0.0742 - auc: 0.9961
Epoch 13/20
32/32 [==============================] - 29s 914ms/step - loss: 0.0210 - auc: 0.9988
Epoch 14/20
32/32 [==============================] - 29s 907ms/step - loss: 0.0015 - auc: 1.0000
Epoch 15/20
32/32 [==============================] - 29s 909ms/step - loss: 5.0718e-04 - auc: 1.0000
Epoch 16/20
32/32 [==============================] - 29s 910ms/step - loss: 3.7656e-04 - auc: 1.0000
Epoch 17/20
32/32 [==============================] - 29s 913ms/step - loss: 3.0531e-04 - auc: 1.0000
Epoch 18/20
32/32 [==============================] - 29s 910ms/step - loss: 2.6734e-04 - auc: 1.0000
Epoch 19/20
32/32 [==============================] - 29s 916ms/step - loss: 2.3592e-04 - auc: 1.0000
Epoch 20/20
32/32 [===

32/32 [==============================] - 27s 851ms/step - loss: 0.0089 - auc: 1.0000
Epoch 16/20
32/32 [==============================] - 27s 850ms/step - loss: 5.0116e-04 - auc: 1.0000
Epoch 17/20
32/32 [==============================] - 27s 849ms/step - loss: 2.6371e-04 - auc: 1.0000
Epoch 18/20
32/32 [==============================] - 27s 848ms/step - loss: 1.6461e-04 - auc: 1.0000
Epoch 19/20
32/32 [==============================] - 27s 846ms/step - loss: 1.5689e-04 - auc: 1.0000
Epoch 20/20
32/32 [==============================] - 27s 848ms/step - loss: 1.2709e-04 - auc: 1.0000
Epoch 1/20
32/32 [==============================] - 31s 950ms/step - loss: 0.6851 - auc: 0.6008
Epoch 2/20
32/32 [==============================] - 31s 956ms/step - loss: 0.4473 - auc: 0.8767
Epoch 3/20
32/32 [==============================] - 30s 952ms/step - loss: 0.1868 - auc: 0.9783
Epoch 4/20
32/32 [==============================] - 31s 958ms/step - loss: 0.0722 - auc: 0.9961
Epoch 5/20
32/32 [========

...

43/43 [==============================] - 74s 2s/step - loss: 0.0189 - auc: 0.9998
Epoch 7/20
43/43 [==============================] - 74s 2s/step - loss: 0.0015 - auc: 1.0000
Epoch 8/20
43/43 [==============================] - 74s 2s/step - loss: 5.0996e-04 - auc: 1.0000
Epoch 9/20
43/43 [==============================] - 74s 2s/step - loss: 2.9716e-04 - auc: 1.0000
Epoch 10/20
43/43 [==============================] - 74s 2s/step - loss: 2.3263e-04 - auc: 1.0000
Epoch 11/20
43/43 [==============================] - 74s 2s/step - loss: 2.1547e-04 - auc: 1.0000
Epoch 12/20
43/43 [==============================] - 74s 2s/step - loss: 1.7429e-04 - auc: 1.0000
Epoch 13/20
43/43 [==============================] - 74s 2s/step - loss: 1.6298e-04 - auc: 1.0000
Epoch 14/20
43/43 [==============================] - 74s 2s/step - loss: 1.4549e-04 - auc: 1.0000
Epoch 15/20
43/43 [==============================] - 74s 2s/step - loss: 1.2609e-04 - auc: 1.0000
Epoch 16/20
43/43 [========================

43/43 [==============================] - 74s 2s/step - loss: 2.9926e-04 - auc: 1.0000
Epoch 12/20
43/43 [==============================] - 74s 2s/step - loss: 2.3128e-04 - auc: 1.0000
Epoch 13/20
43/43 [==============================] - 74s 2s/step - loss: 1.9800e-04 - auc: 1.0000
Epoch 14/20
43/43 [==============================] - 74s 2s/step - loss: 1.7261e-04 - auc: 1.0000
Epoch 15/20
43/43 [==============================] - 74s 2s/step - loss: 1.5343e-04 - auc: 1.0000
Epoch 16/20
43/43 [==============================] - 74s 2s/step - loss: 1.4406e-04 - auc: 1.0000
Epoch 17/20
43/43 [==============================] - 74s 2s/step - loss: 1.2114e-04 - auc: 1.0000
Epoch 18/20
43/43 [==============================] - 73s 2s/step - loss: 1.0936e-04 - auc: 1.0000
Epoch 19/20
43/43 [==============================] - 74s 2s/step - loss: 1.0029e-04 - auc: 1.0000
Epoch 20/20
43/43 [==============================] - 71s 2s/step - loss: 8.9740e-05 - auc: 1.0000
Epoch 1/20
43/43 [==============

43/43 [==============================] - 76s 2s/step - loss: 1.5723e-04 - auc: 1.0000
Epoch 17/20
43/43 [==============================] - 75s 2s/step - loss: 1.4770e-04 - auc: 1.0000
Epoch 18/20
43/43 [==============================] - 76s 2s/step - loss: 1.3449e-04 - auc: 1.0000
Epoch 19/20
43/43 [==============================] - 75s 2s/step - loss: 1.1595e-04 - auc: 1.0000
Epoch 20/20
43/43 [==============================] - 74s 2s/step - loss: 1.1290e-04 - auc: 1.0000
Epoch 1/20
43/43 [==============================] - 77s 2s/step - loss: 0.6476 - auc: 0.6862
Epoch 2/20
43/43 [==============================] - 76s 2s/step - loss: 0.4413 - auc: 0.8766
Epoch 3/20
43/43 [==============================] - 76s 2s/step - loss: 0.1762 - auc: 0.9812
Epoch 4/20
43/43 [==============================] - 76s 2s/step - loss: 0.0583 - auc: 0.9981
Epoch 5/20
43/43 [==============================] - 76s 2s/step - loss: 0.0388 - auc: 0.9988
Epoch 6/20
43/43 [==============================] - 76s 2

43/43 [==============================] - 52s 1s/step - loss: 0.5217 - auc: 0.8074
Epoch 2/20
43/43 [==============================] - 51s 1s/step - loss: 0.2707 - auc: 0.9544
Epoch 3/20
43/43 [==============================] - 51s 1s/step - loss: 0.1717 - auc: 0.9816
Epoch 4/20
43/43 [==============================] - 51s 1s/step - loss: 0.0981 - auc: 0.9938
Epoch 5/20
43/43 [==============================] - 51s 1s/step - loss: 0.0597 - auc: 0.9968
Epoch 6/20
43/43 [==============================] - 51s 1s/step - loss: 0.0296 - auc: 0.9988
Epoch 7/20
43/43 [==============================] - 51s 1s/step - loss: 0.0066 - auc: 0.9999
Epoch 8/20
43/43 [==============================] - 51s 1s/step - loss: 0.0012 - auc: 1.0000
Epoch 9/20
43/43 [==============================] - 51s 1s/step - loss: 5.9141e-04 - auc: 1.0000
Epoch 10/20
43/43 [==============================] - 51s 1s/step - loss: 4.5413e-04 - auc: 1.0000
Epoch 11/20
43/43 [==============================] - 51s 1s/step - loss:

43/43 [==============================] - 78s 2s/step - loss: 0.0320 - auc: 0.9987
Epoch 7/20
43/43 [==============================] - 78s 2s/step - loss: 0.0152 - auc: 0.9998
Epoch 8/20
43/43 [==============================] - 77s 2s/step - loss: 0.0224 - auc: 0.9994
Epoch 9/20
43/43 [==============================] - 78s 2s/step - loss: 0.0042 - auc: 1.0000
Epoch 10/20
43/43 [==============================] - 78s 2s/step - loss: 0.0029 - auc: 1.0000
Epoch 11/20
43/43 [==============================] - 78s 2s/step - loss: 0.0106 - auc: 0.9999
Epoch 12/20
43/43 [==============================] - 78s 2s/step - loss: 0.0110 - auc: 0.9999
Epoch 13/20
43/43 [==============================] - 77s 2s/step - loss: 0.0023 - auc: 1.0000
Epoch 14/20
43/43 [==============================] - 78s 2s/step - loss: 2.2221e-04 - auc: 1.0000
Epoch 15/20
43/43 [==============================] - 77s 2s/step - loss: 1.4558e-04 - auc: 1.0000
Epoch 16/20
43/43 [==============================] - 77s 2s/step - 

## Parallel Computing

In [ ]:
# Parallelizing the computation for all 16 disease conditions

# Parallel(n_jobs=16,backend="threading")(delayed(main_cnn)(x) for x in range(16))
# Parallel(n_jobs=16,backend="threading")(delayed(main_cnn)(x) for x in [0,2,3,4,5,6,7,8,9,10,11,12,13,15])
# Parallel(n_jobs=16,backend="threading")(delayed(main_cnn)(x) for x in [0,1,2])

In [ ]:
time_2 = time.time() - time_1

with open('other/DA_until_balanced_Transformer.csv','a') as fd:
            fd.write(f'Total running time, {time_2}\n')